# CNN 무작정 실행해보기 : notMNIST

## 라이브러리 불러오기

In [ ]:
!wget http://yaroslavvb.com/upload/notMNIST/notMNIST_small.mat

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from torchsummary import summary

from scipy import io

## 데이터셋 불러오기

In [ ]:
data = io.loadmat('notMNIST_small.mat')

data

In [ ]:
x = data['images']
y = data['labels']

In [ ]:
x.shape, y.shape

In [ ]:
classes = 10
resolution = 28

x = np.transpose(x, (2, 0, 1))
x.shape

In [ ]:
x = x.reshape( (-1, 1, resolution, resolution))
x.shape

In [ ]:
# sample, channel, height, width
x.shape, y.shape

* 데이터 살펴보기

In [ ]:
rand_i = np.random.randint(0, x.shape[0])

plt.title( f'idx: {rand_i} , y: {"ABCDEFGHIJ"[ int(y[rand_i]) ]}' )
plt.imshow( x[rand_i, 0, :, :], cmap='gray' )
plt.show()

In [ ]:
rows = 5
fig, axes = plt.subplots(rows, classes, figsize=(classes,rows))

for letter_id in range(classes) :
    letters = x[y==letter_id]      # 0부터 9까지 각 숫자에 맞는 array가 letters에 들어간다.
    letters_len = len(letters)

    for row_i in range(rows) :
        axe = axes[row_i, letter_id]
        axe.imshow( letters[np.random.randint(letters_len), 0, :, :], cmap='gray', interpolation='none')
        axe.axis('off')

plt.show()

## 데이터 전처리

* Data split

    - training set : test set = 8 : 2
    - 재현을 위한 난수 고정 : 2024

In [ ]:
x.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=2024)

In [ ]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

* Scaling

    - min-max scaling

In [ ]:
max_n, min_n = train_x.max(), train_x.min()

In [ ]:
max_n, min_n

In [ ]:
train_x = (train_x - min_n) / (max_n - min_n)
test_x = (test_x - min_n) / (max_n - min_n)

In [ ]:
train_x.max(), train_x.min()

* Numpy -> Torch tensor

In [ ]:
train_x_ts = torch.tensor(train_x, dtype=torch.float32)
test_x_ts = torch.tensor(test_x, dtype=torch.float32)

In [ ]:
train_y_ts = torch.tensor(train_y, dtype=torch.long)
test_y_ts = torch.tensor(test_y, dtype=torch.long)

* Data shape 재확인

In [ ]:
train_x_ts.shape, train_y_ts.shape

## 데이터로더 생성하기
- 데이터셋을 순차적으로 조회 가능하도록 만드는 작업

In [ ]:
from torch.utils.data import TensorDataset

In [ ]:
tr_dataset = TensorDataset(train_x_ts, train_y_ts)
te_dataset = TensorDataset(test_x_ts, test_y_ts)

In [ ]:
for x, y in tr_dataset :
    print(x.shape)
    print(x.dtype)

    print(y.shape)
    print(y.dtype)
    break

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
batch_size = 32

tr_dataloader = DataLoader(dataset=tr_dataset,
                           batch_size=32,
                           )

te_dataloader = DataLoader(dataset=te_dataset,
                           batch_size=32,
                           )

In [ ]:
for x, y in tr_dataloader :
    print('학습 데이터의 형태 [N, C, H, W]:', x.shape)
    print('학습 데이터의 데이터 타입:', x.dtype)

    print('정답 데이터의 형태:', y.shape)
    print('정답 데이터의 데이터 타입:', y.dtype)
    break

## 연산 장치 설정

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

print('연산 장치:', device)

## 모델 정의 및 생성

* 모델의 형태 구성

In [ ]:
class BasicCNN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.basicCNN = nn.Sequential(
            nn.Conv2d(1, 32, (3,3), (1,1), padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2,2), (2,2)),
            nn.Conv2d(32, 64, (3,3), (1,1), padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2,2), (2,2)),
            nn.Flatten(),
            nn.Linear(64*7*7, 10),
        )

    def forward(self, x) :
        x = self.basicCNN(x)
        return x

In [ ]:
model = BasicCNN().to(device)
model

In [ ]:
summary(model, (1, 28, 28))

* 손실 함수와 옵티마이저 설정

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters())

In [ ]:
loss_fn

In [ ]:
optim

* 학습 절차의 함수화

In [ ]:
def train(dataloader, model, loss_fn, optim) :
    model.train()
    size = len(dataloader.dataset)

    for batch, (x, y) in enumerate(dataloader) :
        x, y = x.to(device), y.to(device)

        y_pred = model(x)
        loss = loss_fn(y_pred, y)

        optim.zero_grad()
        loss.backward()
        optim.step()

        if batch % 100 == 0 :
            loss = loss.item()
            current = (batch+1) * len(x)
            print(f'[{current:5d}/{size:5d}] | loss: {loss:.4f}')

In [ ]:
def test(dataloader, model, loss_fn) :
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    test_loss = 0
    correct = 0

    with torch.no_grad() :
        for x, y in dataloader :
            x, y = x.to(device), y.to(device)

            y_pred = model(x)
            test_loss = test_loss + loss_fn(y_pred, y).item()
            correct = correct + (y_pred.argmax()==y).type(torch.float).sum().item()

    test_loss = test_loss / num_batches
    correct = correct / size

    print(f'Accuracy: {100*correct:.2f} | Avg_loss: {test_loss:.4f}')

In [ ]:
epochs = 10

for e in range(epochs) :
    print('Epoch:', e+1)
    print('-----------')

    train(tr_dataloader, model, loss_fn, optim)
    test(te_dataloader, model, loss_fn)
    print('--------------------------')

print('==============================')
print('End')

## 테스트 결과 살펴보기

In [ ]:
classes = ['A','B','C','D','E','F','G','H','I','J']

In [ ]:
len(te_dataset)

In [ ]:
model.eval()

rand_idx = torch.randint(0, 3745, size=(1,)).item()
x, y = te_dataset[rand_idx][0].view((-1,1,28,28)), te_dataset[rand_idx][1]

with torch.no_grad() :
    x = x.to(device)
    y_pred = model(x)

    predicted = y_pred.argmax()
    actual = y

    print(f'Predicted: {classes[predicted]} | Actual: {classes[actual]}')

- 시각화 하여 살펴보기

In [ ]:
model.eval()

figure = plt.figure(figsize=(16, 16))
cols, rows = 5, 5

for i in range(1, cols*rows+1) :
    figure.add_subplot(rows, cols, i)

    rand_idx = torch.randint(0, 3745, size=(1,)).item()
    x, y = te_dataset[rand_idx][0].view((-1,1,28,28)), te_dataset[rand_idx][1]

    with torch.no_grad() :
        x = x.to(device)
        y_pred = model(x)

        predicted = y_pred.argmax()
        actual = y

    plt.title(f'Predicted: {classes[predicted]} | Actual: {classes[actual]}')
    plt.axis("off")
    plt.imshow(x.squeeze(), cmap="gray")

plt.show()

* 틀린 것만 시각화 하여 살펴보기

In [ ]:
model.eval()

predicted_list, actual_list = [], []

for i in range(len(te_dataset)) :
    x, y = te_dataset[i][0].view((-1,1,28,28)), te_dataset[i][1]

    with torch.no_grad() :
        x = x.to(device)
        y_pred = model(x)

        predicted = y_pred.argmax().item()
        actual = y

        predicted_list.append(predicted)
        actual_list.append(actual)

In [ ]:
predicted_list[:5], actual_list[:5]

In [ ]:
false_idx_list = []

for i in range(len(predicted_list)) :
    if predicted_list[i] != actual_list[i] :
        false_idx_list.append(i)

false_idx_list[:5]

In [ ]:
model.eval()

figure = plt.figure(figsize=(16, 16))
cols, rows = 5, 5

for i in range(1, cols*rows+1) :
    figure.add_subplot(rows, cols, i)

    sample_idx = false_idx_list[ torch.randint(len(false_idx_list), size=(1,)).item() ]

    with torch.no_grad() :
        temp_x = te_dataset[ sample_idx ][0].view((-1,1,28,28))
        temp_y_pred = model(temp_x)

        temp_predicted = temp_y_pred.argmax().item()
        temp_actual = te_dataset[ sample_idx ][1]

    plt.title(f'Predicted: {classes[temp_predicted]} | Actual: {classes[temp_actual]}')
    plt.axis("off")
    plt.imshow(temp_x.squeeze(), cmap="gray")

plt.show()